# Problema 1

In [30]:
import os
import numpy as np
from PIL import Image

# Define el directorio donde se encuentran las imágenes y dónde guardarlas
input_directory = './database'
output_directory = './database'

# Crea el directorio de salida si no existe
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Procesa y aplica un umbral a cada imagen
for i in range(1, 21):  # Suponiendo que las imágenes están numeradas del 1 al 20
    # Construye el nombre de archivo
    file_namef = f'{i}.pgm'
    file_name = f'{i}'
    input_path = os.path.join(input_directory, file_namef)
    output_path = os.path.join(output_directory, f'{file_name}_th.pgm')

    # Realiza la misma operación que antes, pero en un bucle para cada imagen
    try:
        # Carga y convierte la imagen a escala de grises
        image = Image.open(input_path).convert("L")
        image_np = np.array(image)

        # Aplica un umbral
        thresholded_np = np.where(image_np < 65, 250, image_np)
        thresholded_image = Image.fromarray(thresholded_np.astype(np.uint8))

        # Guarda la imagen umbralizada
        thresholded_image.save(output_path)
    except FileNotFoundError:
        print(f"Archivo {input_path} no encontrado. Saltando.")

# Devuelve la ruta al directorio que contiene las imágenes umbralizadas
output_directory


'./database'

In [28]:
import cv2

def invert_colors(img):
    # Invertir los colores de la imagen
    return 255 - img

def bradley_roth_adaptive_threshold(image, s=15, t=0.07):
    # Convertir imagen a escala de grises si es necesario
    if image.mode != 'L':
        image = image.convert('L')
    img = np.array(image, dtype=np.float64)

    # Calcular las sumas integrales
    integral_img = np.cumsum(np.cumsum(img, axis=1), axis=0)

    # Definir tamaño de la ventana
    rows, cols = img.shape
    s2 = s // 2
    result = np.zeros_like(img)

    for i in range(rows):
        for j in range(cols):
            # Definir límites de la ventana
            x1 = max(j - s2, 0)
            x2 = min(j + s2, cols - 1)
            y1 = max(i - s2, 0)
            y2 = min(i + s2, rows - 1)

            count = (y2 - y1) * (x2 - x1)
            sum_ = integral_img[y2, x2] - integral_img[y1, x2] - integral_img[y2, x1] + integral_img[y1, x1]

            if img[i, j] * count <= sum_ * (1 - t):
                result[i, j] = 0
            else:
                result[i, j] = 255

    # Invertir colores justo antes de retornar la imagen
    result = invert_colors(result)
    return Image.fromarray(result.astype(np.uint8))

def process_images_in_folder(folder_path):
    for i in range(1, 21):  # Suponiendo 20 imágenes
        original_path = os.path.join(folder_path, f'{i}_th.pgm')
        
        if os.path.exists(original_path):
            image = Image.open(original_path)
            binarized_image = bradley_roth_adaptive_threshold(image)
            binarized_image.save(os.path.join(folder_path, f'{i}_bn.pgm'))
            print(f'Imagen procesada y guardada: {i}_bn.pgm')
        else:
            print(f'Imagen no encontrada: {original_path}')

# Actualizar con la ruta de tu carpeta
folder_path = './database'
process_images_in_folder(folder_path)


Imagen procesada y guardada: 1_bn.pgm
Imagen procesada y guardada: 2_bn.pgm
Imagen procesada y guardada: 3_bn.pgm
Imagen procesada y guardada: 4_bn.pgm
Imagen procesada y guardada: 5_bn.pgm
Imagen procesada y guardada: 6_bn.pgm
Imagen procesada y guardada: 7_bn.pgm
Imagen procesada y guardada: 8_bn.pgm
Imagen procesada y guardada: 9_bn.pgm
Imagen procesada y guardada: 10_bn.pgm
Imagen procesada y guardada: 11_bn.pgm
Imagen procesada y guardada: 12_bn.pgm
Imagen procesada y guardada: 13_bn.pgm
Imagen procesada y guardada: 14_bn.pgm
Imagen procesada y guardada: 15_bn.pgm
Imagen procesada y guardada: 16_bn.pgm
Imagen procesada y guardada: 17_bn.pgm
Imagen procesada y guardada: 18_bn.pgm
Imagen procesada y guardada: 19_bn.pgm
Imagen procesada y guardada: 20_bn.pgm


In [29]:
import numpy as np
import cv2
import os

def calculate_metrics(gt, pred):
    TP = np.sum((gt == 255) & (pred == 255))
    TN = np.sum((gt == 0) & (pred == 0))
    FP = np.sum((gt == 0) & (pred == 255))
    FN = np.sum((gt == 255) & (pred == 0))

    accuracy = (TP + TN) / (TP + TN + FP + FN)
    sensitivity = TP / (TP + FN) if (TP + FN) != 0 else 0
    specificity = TN / (TN + FP) if (TN + FP) != 0 else 0
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    f1_score = 2*TP / (2*TP + FP + FN) if (2*TP + FP + FN) != 0 else 0

    return accuracy, sensitivity, specificity, precision, f1_score

folder_path = './database'

metrics = []

for i in range(1, 21):
    gt_path = os.path.join(folder_path, f'{i}_gt.pgm')
    bn_path = os.path.join(folder_path, f'{i}_bn.pgm')
    
    if os.path.exists(gt_path) and os.path.exists(bn_path):
        gt = cv2.imread(gt_path, 0)
        pred = cv2.imread(bn_path, 0)
        
        metrics.append(calculate_metrics(gt, pred))
    else:
        metrics.append((None, None, None, None, None))

print("{:<6} | {:<9} | {:<11} | {:<11} | {:<9} | {:<7}".format("Image", "Accuracy", "Sensitivity", "Specificity", "Precision", "F1 Score"))
print("-" * 70)  # Imprime una línea separadora

for i, (accuracy, sensitivity, specificity, precision, f1_score) in enumerate(metrics, 1):
    print("{:<6} | {:<9.3f} | {:<11.3f} | {:<11.3f} | {:<9.3f} | {:<7.3f}".format(i, accuracy or 0, sensitivity or 0, specificity or 0, precision or 0, f1_score or 0))


Image  | Accuracy  | Sensitivity | Specificity | Precision | F1 Score
----------------------------------------------------------------------
1      | 0.917     | 0.517       | 0.935       | 0.258     | 0.344  
2      | 0.941     | 0.579       | 0.959       | 0.411     | 0.481  
3      | 0.938     | 0.516       | 0.957       | 0.343     | 0.412  
4      | 0.957     | 0.131       | 0.996       | 0.581     | 0.213  
5      | 0.913     | 0.602       | 0.925       | 0.231     | 0.334  
6      | 0.930     | 0.725       | 0.938       | 0.306     | 0.430  
7      | 0.832     | 0.309       | 0.860       | 0.102     | 0.153  
8      | 0.896     | 0.322       | 0.924       | 0.172     | 0.224  
9      | 0.908     | 0.657       | 0.922       | 0.311     | 0.422  
10     | 0.941     | 0.369       | 0.968       | 0.343     | 0.355  
11     | 0.928     | 0.536       | 0.945       | 0.293     | 0.379  
12     | 0.947     | 0.639       | 0.974       | 0.677     | 0.657  
13     | 0.913     | 0.703     

1. Accuracy
Qué mide: La proporción de predicciones correctas (tanto positivas como negativas) entre el total de casos.
Valores buenos: Valores cercanos a 1 (o 100%) son ideales, indicando que la mayoría de las predicciones son correctas.
2. Sensitivity (Recall)
Qué mide: La proporción de verdaderos positivos identificados correctamente de todos los casos positivos reales.
Valores buenos: Valores cercanos a 1 son mejores, lo que significa que el algoritmo es capaz de identificar correctamente la mayoría de los casos positivos reales.
3. Specificity
Qué mide: La proporción de verdaderos negativos identificados correctamente de todos los casos negativos reales.
Valores buenos: Al igual que con la Sensitivity, valores cercanos a 1 son ideales, indicando una alta capacidad para identificar correctamente los casos negativos.
4. Precision
Qué mide: La proporción de verdaderos positivos sobre el total de positivos predichos (verdaderos positivos + falsos positivos).
Valores buenos: Valores más altos (cercanos a 1) son deseables, lo que indica que cuando el modelo predice un caso positivo, es probable que sea correcto.
5. F1 Score
Qué mide: El promedio armónico de Precision y Sensitivity, ofreciendo un balance entre ambas.
Valores buenos: Un F1 Score cercano a 1 es excelente, indicando un equilibrio óptimo entre Precision y Sensitivity.